In [ ]:
import pyroSAR
from pyroSAR.snap.auxil import parse_recipe, parse_node
from pyroSAR.snap.auxil import gpt, groupbyWorkers

from os.path import join, exists
from os import makedirs
from shutil import rmtree
from glob import glob

from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
def create_coherence_gpt(in_slc_one, in_slc_two, dem_path, out_filename, out_graph_path, split_params, **kwargs):
    
    workflow = parse_recipe('blank')
    
    # 1st branch
    read_one = parse_node('Read')
    read_one.parameters['file'] = in_slc_one
    workflow.insert_node(read_one)
    
    topsar_split_one = parse_node('TOPSAR-Split')
    topsar_split_one.parameters['subswath'] = split_params['subswath']
    topsar_split_one.parameters['firstBurstIndex'] = split_params['burst_first']
    topsar_split_one.parameters['lastBurstIndex'] = split_params['burst_last']
    workflow.insert_node(topsar_split_one, before=read_one.id)
    """
    apply_orbit_one = parse_node('Apply-Orbit-File')
    apply_orbit_one.parameters['orbitType'] = 'Sentinel Precise (Auto Download)'
    apply_orbit_one.parameters['polyDegree'] = 3
    # WARNING - the script currently ignores orbit file application if it is not found
    apply_orbit_one.parameters['continueOnFail'] = 'true'
    workflow.insert_node(apply_orbit_one, before=topsar_split_one.id)
    """
    # 2nd branch
    read_two = parse_node('Read')
    read_two.parameters['file'] = in_slc_two
    workflow.insert_node(read_two)
    
    topsar_split_two = parse_node('TOPSAR-Split')
    topsar_split_two.parameters['subswath'] = split_params['subswath']
    topsar_split_two.parameters['firstBurstIndex'] = split_params['burst_first']
    topsar_split_two.parameters['lastBurstIndex'] = split_params['burst_last']
    workflow.insert_node(topsar_split_two, before=read_two.id)
    """
    apply_orbit_two = parse_node('Apply-Orbit-File')
    apply_orbit_two.parameters['orbitType'] = 'Sentinel Precise (Auto Download)'
    apply_orbit_two.parameters['polyDegree'] = 3
    # WARNING - the script currently ignores orbit file application if it is not found
    apply_orbit_two.parameters['continueOnFail'] = 'true'
    workflow.insert_node(apply_orbit_two, before=topsar_split_two.id)
    """
    # combined branch
    back_geocoding = parse_node('Back-Geocoding')
    back_geocoding.parameters['demName'] = 'External DEM'
    back_geocoding.parameters['demResamplingMethod'] = 'BILINEAR_INTERPOLATION'
    back_geocoding.parameters['externalDEMFile'] = dem_path
    back_geocoding.parameters['externalDEMNoDataValue'] = 0.0
    back_geocoding.parameters['resamplingType'] = 'BILINEAR_INTERPOLATION'
    back_geocoding.parameters['maskOutAreaWithoutElevation'] = 'false'
    back_geocoding.parameters['outputRangeAzimuthOffset'] = 'false'
    back_geocoding.parameters['outputDerampDemodPhase'] = 'false'
    back_geocoding.parameters['disableReramp'] = 'false'
    #workflow.insert_node(back_geocoding, before=[apply_orbit_one.id, apply_orbit_two.id])
    workflow.insert_node(back_geocoding, before=[topsar_split_one.id, topsar_split_two.id])

    coherence = parse_node('Coherence')
    coherence.parameters['cohWinAz'] = 2
    coherence.parameters['cohWinRg'] = 10
    #coherence.parameters['substractFlatEarthPhase'] = 'false'
    coherence.parameters['srpPolynomialDegree'] = 5
    coherence.parameters['srpNumberPoints'] = 501
    coherence.parameters['orbitDegree'] = 3
    coherence.parameters['squarePixel'] = 'true'
    coherence.parameters['subtractTopographicPhase'] = 'false'
    coherence.parameters['demName'] = 'External DEM'
    #coherence.parameters['demResamplingMethod'] = 'BILINEAR_INTERPOLATION'
    coherence.parameters['externalDEMFile'] = dem_path
    coherence.parameters['externalDEMNoDataValue'] = 0.0
    coherence.parameters['externalDEMApplyEGM'] = 'true'
    coherence.parameters['tileExtensionPercent'] = 100
    coherence.parameters['singleMaster'] = 'true'
    workflow.insert_node(coherence, before=back_geocoding.id)

    deburst = parse_node('TOPSAR-Deburst')
    workflow.insert_node(deburst, before=coherence.id)

    tc = parse_node('Terrain-Correction')
    tc.parameters['demName'] = 'External DEM'
    tc.parameters['demResamplingMethod'] = 'BILINEAR_INTERPOLATION'
    tc.parameters['externalDEMFile'] = dem_path
    tc.parameters['externalDEMNoDataValue'] = 0.0
    tc.parameters['externalDEMApplyEGM'] = 'true'

    tc.parameters['imgResamplingMethod'] = 'BILINEAR_INTERPOLATION'
    tc.parameters['pixelSpacingInMeter'] = 0.0
    tc.parameters['pixelSpacingInDegree'] = 0.0
    tc.parameters['mapProjection'] = '''GEOGCS["WGS 84", DATUM["World Geodetic System 1984",
        SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], 
        AUTHORITY["EPSG","6326"]], 
        PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], 
        UNIT["degree", 0.017453292519943295], 
        AXIS["Geodetic longitude", EAST], 
        AXIS["Geodetic latitude", NORTH], 
        AUTHORITY["EPSG","4326"]]'''
    # maybe? so everything is aligned
    tc.parameters['alignToStandardGrid'] = 'true'
    tc.parameters['standardGridOriginX'] = 0.0
    tc.parameters['standardGridOriginY'] = 0.0
    tc.parameters['nodataValueAtSea'] = 'false'
    tc.parameters['saveDEM'] = 'false'
    tc.parameters['saveLatLon'] = 'false'
    tc.parameters['saveIncidenceAngleFromEllipsoid'] = 'false'
    tc.parameters['saveLocalIncidenceAngle'] = 'false'
    tc.parameters['saveProjectedLocalIncidenceAngle'] = 'false'
    tc.parameters['saveSelectedSourceBand'] = 'true'
    tc.parameters['saveLayoverShadowMask'] = 'false'
    #tc.parameters['outputComplex'] = 'false'
    tc.parameters['applyRadiometricNormalization'] = 'false'
    tc.parameters['saveSigmaNought'] = 'false'
    tc.parameters['saveGammaNought'] = 'false'
    tc.parameters['saveBetaNought'] = 'false'
    tc.parameters['incidenceAngleForSigma0'] = 'Use projected local incidence angle from DEM'
    tc.parameters['incidenceAngleForGamma0'] = 'Use projected local incidence angle from DEM'
    tc.parameters['auxFile'] = 'Latest Auxiliary File'
    workflow.insert_node(tc, before=deburst.id)

    write = parse_node('Write')
    write.parameters['file'] = out_filename
    write.parameters['formatName'] = 'GeoTIFF-BigTIFF'
    workflow.insert_node(write, before=tc.id)

    workflow.write(out_graph_path)

### Run processing in a loop

In [ ]:
# Define directiories used
ROOT_DIR = 'd:\\studenti\\JD\\DP'

dtm_path = join(ROOT_DIR, 'DTM\\DMR_4G_4326.tif')
temp_dir = join(ROOT_DIR, 'temp')
xmls_dir = join(temp_dir, 'snap_xmls')
makedirs(xmls_dir) if not exists(xmls_dir) else print(f'{xmls_dir} already exists.')

In [ ]:
# create list of data directories to process and corresponding list of parameters to the TOPSAR_split function
data_dir_list = (join(ROOT_DIR, 's1_download\\2021\\146'), )
out_dir_list = (join(ROOT_DIR, 's1_coherence\\2021\\146'), )
split_params_list = ({
    'subswath': 'IW3',
    'burst_first': 2,
    'burst_last': 3}, )
data_dir_list

In [ ]:
for data_dir, out_dir, split_params in zip(data_dir_list, out_dir_list, split_params_list):
    files_list = glob('S1*.zip', root_dir=data_dir)
    files_list.sort(key=lambda x : x[17:25])
    dates_list = (filename[17:25] for filename in files_list)
    
    for idx in range(len(files_list)-1):
        date_1, date_2 = files_list[idx][17:25], files_list[idx+1][17:25]
        print(f'Computing coherence between images from {date_1} and {date_2}')
    
        graph_path = join(xmls_dir, f'coherence_{date_1}_{date_2}.xml')
        in_filepath_one = join(data_dir, files_list[idx])
        in_filepath_two = join(data_dir, files_list[idx+1])
        out_path = join(out_dir, f'coh_{date_1}_{date_2}')
    
        create_coherence_gpt(in_filepath_one, in_filepath_two, dtm_path, out_path, graph_path, split_params)
        gpt(xmlfile=graph_path, tmpdir=temp_dir, groups=groupbyWorkers(graph_path, n=2))

#rmtree(temp_dir)

## Possibly no longer needed?

In [ ]:
dem_filepath = 'F:/datasets/NATUR_CUNI/DEM/DMR_4G_4326.tif'
workflow_dir = 'C:/users/dd/documents/natur_cuni/_dp/code_git_repo/snap_scripts'
DATA_DIR = 'F:/datasets/NATUR_CUNI/_dp_download/sentinel1/146'
out_dir = 'F:/datasets/NATUR_CUNI/_dp_coherence/2021/146'

In [ ]:
files_list = glob('S1*.zip', root_dir=DATA_DIR)
dates_list = [filename[17:25] for filename in files_list]

In [ ]:
files_list.sort(key=lambda x : x[17:25])
files_list

In [ ]:
for idx in range(len(files_list)-1):
    date_1, date_2 = files_list[idx][17:25], files_list[idx+1][17:25]
    print(f'Computing coherence between images from {date_1} and {date_2}')
    
    graph_path = join(workflow_dir, f'coherence_{date_1}_{date_2}.xml')
    in_filepath_one = join(DATA_DIR, files_list[idx])
    in_filepath_two = join(DATA_DIR, files_list[idx+1])
    out_filepath = join(out_dir, f'coh_{date_1}_{date_2}')
    
    create_coherence_gpt(in_filepath_one, in_filepath_two, dem_filepath, out_filepath, graph_path)
    gpt(graph_path, DATA_DIR, groups=groupbyWorkers(graph_path, n=2))

### Show capture dates

In [ ]:
dates_list.sort()
print(dates_list)

In [ ]:
plt.plot([datetime.strptime(i, '%Y%m%d') for i in dates_list], len(dates_list)*[0], '+')